# Imports

In [36]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.1 MB/s eta 0:00:00


In [37]:
# From Imports
from math import log, sqrt, pi, exp
from scipy.stats import norm
from datetime import datetime, date
from pandas import DataFrame
from datetime import datetime
from scipy.optimize import minimize
from arch import arch_model
from sklearn.linear_model import LinearRegression

# Alias Imports
import numpy_financial as npf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Imports
import math
import arch
import openpyxl
import pprint
import pmdarima as pm

print("Import Complete")

Import Complete


# Importing Bond Trading Data

In [10]:
# Importing Excel data of 2Y and 10Y Tbills
tbill2Y = pd.read_excel("Treasury_Data_V1.xlsx", "T 4.625 02 28 25 Govt TRADES")
tbill10Y = pd.read_excel("Treasury_Data_V1.xlsx", "T 3.375 05 15 33 Govt TRADES")
#note, volume is traded in increments of $1000 USD at STGT Exchange
#note, need to clean 10Y data, bids / asks don't line up

# Making Trade Time, Trade Volume, and Ask Time into lists for 2Y Bond Data
dates2Y = list(tbill2Y["Trade Time"])
asks2Y = list(tbill2Y["Ask Time"]) # Currently Not Used

print("Excel Data Imported")

Excel Data Imported


# Cleaning Bond Trading Data

In [11]:
# Cleaning Trade Time data
clean_dates2Y = []

# Going through the range and converting dates to Numpy datetime64s
for i in range(len(dates2Y)):
    date = np.datetime64(str(dates2Y[i].date()))

    # Removing all NaT values from the list
    if not np.isnat(date):

        # Adding all dates to our clean_dates2Y list
        clean_dates2Y.append(date)

    # If no date given, we make datetime max value for easy spotting
    else:
        clean_dates2Y.append(np.datetime64(datetime.max))

# Cleaning Ask Time data
clean_asks2Y = []

# Going through the range and converting dates to Numpy datetime64s
for i in range(len(asks2Y)):
    date = np.datetime64(str(asks2Y[i].date()))

    # Removing all NaT values from the list
    if not np.isnat(date):

        # Adding all dates to our clean_asks2Y list
        clean_asks2Y.append(date)

    # If no date given, we make datetime max value for easy spotting
    else:
        clean_dates2Y.append(np.datetime64(datetime.max))

print("Data cleaned")

Data cleaned


# Calculating Average Daily Volume

In [13]:
# Adding new Date object column to data
tbill2Y['Trade Time Date'] = clean_dates2Y
tbill2Y['Ask Time Date'] = clean_asks2Y

# Calculating the total volume on each specific day
day_volumes = tbill2Y.groupby(['Trade Time Date'])['Trade Volume'].sum()

# Getting all the unique dates from Trade Time Date
unique_dates = tbill2Y['Trade Time Date'].unique()

# Getting rid of the last row
minus_max_date = np.resize(unique_dates, unique_dates.size - 1)
minus_max_volume = np.resize(day_volumes, day_volumes.size - 1)

# Creating Average Hourly Trading Volumes from Daily Volumes in Dictionary
average_volume_daily = {} #key is date, value is total_volume

# Creating dictionary around unique dates, and summed daily volumes
for day, volume in zip(minus_max_date, day_volumes):

    # Calculate the average volume for the date and store it in the dictionary
    average_volume_daily[day] = volume / 12.0

# Splitting into lists of Dates, Volumes
dates = average_volume_daily.keys()
hourly_volumes = average_volume_daily.values()

# Formatting Float in Pandas
pd.options.display.float_format = '{:.2f}'.format

# Print the resulting list
daily_volumes = pd.DataFrame()
daily_volumes['Trading Day'] = dates
daily_volumes['Hourly Volume'] = hourly_volumes

print("Daily Volumes")

Daily Volumes


# Simulating Price and Spread Process

In [14]:
#1) Use Cox-Ingersoll-Ross Model (CIR) model to simulate bond yield process (approximation of price process)
#2) Use GARCH model to simulate bid-ask spreads in one step ahead forecasts
#3) Z is simulated price, ask is Zu, bid is Zl, utilize to calculate daily portfolio value and loss
#4) Calculate compounded interest loss
#5) Add final portfolio loss to interest loss, print resuslts of all simulations
#OTHER NOTES#
#determine what k value we want to keep, assume constant k, $50B issue size

# Calculate instantaneous LP position value as a function of k, Zl (bid), Zu (ask), and Z (price)

In [15]:
def alpha_t(k, Z, Zl, Zu):
    Z = np.where(Z < 0, np.nan, Z)
    Zl = np.where(Zl < 0, np.nan, Zl)
    Zu = np.where(Zu < 0, np.nan, Zu)

    xi = k * (np.sqrt(Z) - np.sqrt(Zl))
    xi = np.where(np.isnan(xi), 0, xi)

    yi = k * (np.sqrt(Zu) - np.sqrt(Zl))
    yi = np.where(np.isnan(yi), 0, yi)

    return xi + yi * Z

# Calculating Historical Midpoint between Bid and Ask

In [16]:
historical_bid_data = tbill2Y["Bid"]
historical_ask_data = tbill2Y["Ask"]
historical_midpoint = (historical_bid_data + historical_ask_data) / 2
historical_spread = abs(historical_bid_data - historical_ask_data)
# historical_midpoint

print("Calculated Historical Midpoint and Historical Spreads")

Calculated Historical Midpoint and Historical Spreads


# Calculating Price to Yield

In [17]:
def price_to_yield(price, face_value, coupon_rate, coupon_frequency, current_date, maturity_date):
    time_to_maturity_days = (maturity_date - current_date).days
    time_to_maturity_hours = time_to_maturity_days * 24  # Convert to hours

    full_coupon_periods = int(time_to_maturity_hours / (365.0 * 24 / coupon_frequency))
    partial_coupon_period_hours = time_to_maturity_hours % (365.0 * 24 / coupon_frequency)

    cash_flows = [(coupon_rate * face_value) / coupon_frequency] * full_coupon_periods

    # Handle the partial coupon period
    if partial_coupon_period_hours > 0:
        partial_coupon_payment = (coupon_rate * face_value * partial_coupon_period_hours) / (365.0 * 24)
        cash_flows.append(partial_coupon_payment)

    cash_flows[-1] += face_value  # Add the face value at maturity
    yield_value = np.nan

    try:
        yield_value = npf.irr([-price] + cash_flows)
    except ValueError:
        pass

    return yield_value

# Estimate CIR model parameters using historical yields

In [18]:
def cir_likelihood(parameters, data):
    mu, sigma, kappa, theta = parameters
    dt = 1 / (252.0 * 12)  # Hourly data assumed
    n = len(data)
    log_likelihood = 0.0

    for i in range(1, n):
        Zl = data[i - 1]
        Z = data[i]
        Z_diff = Z - Zl
        gamma = np.sqrt(kappa**2 + 2 * sigma**2)

        log_likelihood += (
            -(n - 1) * (np.log(2 * gamma) - np.log(sigma))
            - (kappa + gamma) * Z_diff
            - 2 * np.log((2 * gamma * np.exp(kappa + gamma * dt)) / (2 * gamma + (kappa + gamma) * (np.exp(gamma * dt) - 1)))
        )

    return -log_likelihood

initial_parameters = [0.05, 0.1, 0.2, 0.03]  # Initial guesses for parameters
result = minimize(cir_likelihood, initial_parameters, args=(historical_midpoint,), method='L-BFGS-B')
mu, sigma, kappa, theta = result.x
print(result.x)

[ 0.05        1.05484352 -0.00940215  0.03      ]


# Simulate bond yields based on the estimated CIR parameters

In [107]:
T = 1  # Time to maturity, years
n_simulations = 1  # Num of sims
n_periods = 252 * 12 # Num of hourly periods in a year
coupon_rate = 0.04625
coupon_frequency = 2

simulated_yields = np.zeros((n_simulations, n_periods)) #init array
bond_prices = np.zeros((n_simulations, n_periods)) #init array

for i in range(n_simulations):
    dt = T / n_periods
    current_date = tbill2Y["Bid Time"][0]
    maturity_date = current_date + pd.DateOffset(years=T)
    bond_price = historical_midpoint.iloc[0]  # Initial bond price at first midpoint value

    for j in range(1, n_periods):
        bond_yield = price_to_yield(bond_price, 1000, coupon_rate, coupon_frequency, current_date, maturity_date)
        gamma = np.sqrt(kappa**2 + 2 * sigma**2)
        bond_yield += kappa * (theta - bond_yield) * dt + sigma * np.sqrt(bond_yield) * np.random.normal(0, np.sqrt(dt))  # CIR process
        bond_price = 1000 / ((1 + bond_yield / coupon_frequency) ** (coupon_frequency * (maturity_date - current_date).days / (365 * 24)))  # Convert yield to price
        bond_prices[i, j] = bond_price
        simulated_yields[i, j] = bond_yield

cols = ["sim_bond_prices", "sim_volume", "sim_spread"]
sim_df = pd.DataFrame(columns=cols)

bond_prices_t = bond_prices.transpose()

for i in range(bond_prices_t.shape[0]):
  sim_df.loc[i,'sim_bond_prices'] = bond_prices_t[i]

print(sim_df)

3024
          sim_bond_prices sim_volume sim_spread
0                   [0.0]        NaN        NaN
1     [959.7814963891697]        NaN        NaN
2     [999.1389744597064]        NaN        NaN
3     [999.2643864161195]        NaN        NaN
4     [999.3949744787324]        NaN        NaN
...                   ...        ...        ...
3019  [999.3852575188167]        NaN        NaN
3020  [999.2502151855607]        NaN        NaN
3021    [999.28158123175]        NaN        NaN
3022  [999.2347303449036]        NaN        NaN
3023  [999.2094694356964]        NaN        NaN

[3024 rows x 3 columns]


# Simulate Risk Free Rate Yields

In [82]:
T = 1  # Time to maturity, years
n_simulations = 1  # Num of sims
n_periods = 252 * 12 # Num of hourly periods in a year
coupon_rate = 0.04625
coupon_frequency = 2

simulated_yields_risk_free_rate = np.zeros((n_simulations, n_periods)) #init array

for i in range(n_simulations):
    dt = T / n_periods
    current_date = tbill2Y["Bid Time"][0]
    maturity_date = current_date + pd.DateOffset(years=T)

    for j in range(1, n_periods):
        bond_yield = .0525
        gamma = np.sqrt(kappa**2 + 2 * sigma**2)
        bond_yield += kappa * (theta - bond_yield) * dt + sigma * np.sqrt(bond_yield) * np.random.normal(0, np.sqrt(dt))  # CIR process
        simulated_yields[i, j] = bond_yield

print("Simulated Yields Risk Free Rate")

Simulated Yields Risk Free Rate


# Estimating OU (Ornstein–Uhlenbeck) Process Paramters for Spread Simulation

In [62]:
# Historical spread data is used from above calculation
# Function to calculate the OU likelihood
def ou_likelihood(parameters, data):
    mean_reversion, vol, initial_spread_min = parameters
    dt = 1  # Time step (you can adjust this based on your data frequency)
    log_likelihood = 0.0
    initial_spread = initial_spread_min

    for i in range(1, len(historical_spread)):
        spread_diff = historical_spread[i] - initial_spread
        log_likelihood += -0.5 * (spread_diff / vol) ** 2
        log_likelihood -= 0.5 * np.log(2 * np.pi * vol ** 2 * dt)
        initial_spread += mean_reversion * (initial_spread_min - initial_spread) * dt

    return -log_likelihood

# Initial parameter guesses
intial_mean_reversion = 0.1
intial_vol = 0.05
intial_min_arg = historical_spread[0]

# Packing them into parameter tupe
initial_parameters = (intial_mean_reversion, intial_vol, intial_min_arg)

# Minimize the negative log-likelihood to estimate OU parameters
result = minimize(ou_likelihood, initial_parameters, args=(historical_spread,), method='L-BFGS-B')

# Extract estimated parameters
mean_reversion, volatility, initial_spread = result.x

print("OU Likelihood")

OU Likelihood


# Simulating Spreads with OU Process

In [121]:
n_simulations = 1  # Num of sims
n_periods = 252 * 12 # Num of hourly periods in a year

simulated_spreads = np.zeros((n_simulations, n_periods)) #init array

for i in range(n_simulations):
    spread = initial_spread

    for j in range(1, n_periods):
        spread += mean_reversion * (initial_spread - spread) * dt + volatility * np.sqrt(dt) * np.random.normal(0, 1)
        simulated_spreads[i, j] = spread

simulated_spreads_t = simulated_spreads.transpose()

for i in range(simulated_spreads_t.shape[0]):
     sim_df.loc[i,'sim_spread'] = simulated_spreads_t[i]

print(sim_df["sim_spread"])
print("Spreads Simulated with OU")

1
0                        [0.0]
1       [0.010729871219321539]
2       [0.010663801178663265]
3       [0.010761236907522658]
4       [0.010607005977475851]
                 ...          
3019    [0.009886427665827091]
3020    [0.009824659546069004]
3021    [0.009852644546090163]
3022    [0.009905729030555916]
3023    [0.010053077169002944]
Name: sim_spread, Length: 3024, dtype: object
Spreads Simulated with OU


# Simulating Volume with Poisson Process

In [122]:
# Replace with volume data
# Calculate the average trading rate (λ) from historical data

hourly_periods = 252 * 12 # Hourly Trading Periods

# Calculate sums of Hourly Volume
total_historical_sum = 0
for hourly_volume in hourly_volumes:
    total_historical_sum += hourly_volume
average_historical_trading_rate = total_historical_sum / len(hourly_volumes)

# Simulate trading volume using a Poisson process
simulated_hourly_volume = np.random.poisson(average_historical_trading_rate, hourly_periods)

# Calculate sums of Hourly Volume
total_simulated_sum = 0
for hourly_volume in simulated_hourly_volume:
    total_simulated_sum += hourly_volume
average_simulated_trading_rate = total_simulated_sum / len(simulated_hourly_volume)

# Adjust for daily variation (for example, increase trading rate during high-activity hours)
# Can customize this adjustment based on your market's characteristics
simulated_hourly_volume[0] *= 1.2  # Increase volume during market open
simulated_hourly_volume[-1] *= 1.2  # Increase volume during market close

# Normalize simulated volume to match historical data
np.multiply(simulated_hourly_volume, total_historical_sum / total_simulated_sum)

# Create a DataFrame to store the simulated volume data
simulated_data = pd.DataFrame({'Hourly_Volume': simulated_hourly_volume})

for i in range(simulated_hourly_volume.shape[0]):
     sim_df.loc[i,'sim_volume'] = simulated_hourly_volume[i]

print(sim_df)

          sim_bond_prices sim_volume              sim_spread
0                   [0.0]   44178147                   [0.0]
1     [959.7814963891697]   36811650  [0.010729871219321539]
2     [999.1389744597064]   36820656  [0.010663801178663265]
3     [999.2643864161195]   36823245  [0.010761236907522658]
4     [999.3949744787324]   36822686  [0.010607005977475851]
...                   ...        ...                     ...
3019  [999.3852575188167]   36814294  [0.009886427665827091]
3020  [999.2502151855607]   36818576  [0.009824659546069004]
3021    [999.28158123175]   36819260  [0.009852644546090163]
3022  [999.2347303449036]   36812772  [0.009905729030555916]
3023  [999.2094694356964]   44186568  [0.010053077169002944]

[3024 rows x 3 columns]


# Predicted Loss Functions



In [32]:
def alpha_t(k, Z, Zl, Zu):
        if Zl < Z < Zu:
            xi = k * (Z**(1/2) - (Zl)**(1/2))
            yi = k * (Z**(-1/2) - Zu**(-1/2))
            alpha = xi + yi * Z
        elif Z < Zl:
            xi = 0
            yi = k * ((Zl)**(-1/2) - (Zu)**(-1/2))
            alpha = xi + yi * Z
        elif Z > Zu:
            xi = k * ((Zu)**(1/2) - (Zl)**(1/2))
            yi = 0
            alpha = xi + yi * Z
        return alpha

#See denominator or convexity cost
def LP_spread_conversion(Z, Zl, Zu):
    sigma_u = -2 * (Z**(1/2)) / (Zu**(1/2)) + 2
    sigma_l = -2 * (Z**(1/2)) / (Zl**(1/2)) + 2
    return sigma_u + sigma_l

#Quadratic Variation, needs multiple step ahead forecast, maybe generate 20 forecasts over the short block interval
def calc_QV(Z):
    log_returns = np.diff(np.log(price_list))
    QV = []
    for i in range(10, len(log_returns), 10):
        QV.append(np.sum(np.diff(log_returns[i-10:i])**2))
    QV = pd.Series(QV)
    return QV

def calculate_conv_cost(alpha_lu, Zl, Zu, Z):
    integral = 0
    for i in range(len(Z)):
      alpha_curr_lu = alpha_lu[i]
      spread = LP_spread_conversion(Z[i], Zl_avg[i], Zu_avg[i])
      quad_var = calc_QV(Z[:i])
      integral += alpha_curr_lu / (2* spread * Z[i]**2) * quad_var

    return integral


def calculate_opportunity_cost(alpha_0, alpha_d, risk_free_rate_data, dt_values):
    opportunity_cost = 0.0

    for alpha, r, dt in zip(alpha_d, risk_free_rate_data, dt_values):
        opportunity_cost += (alpha_0 - alpha) * r * dt

    return opportunity_cost

# Calculating Fee Policy Using Forecasting

In [145]:
#NOTE, EDITED USING ROLLING ORIGIN VALIDATION (ROV)

# Define the number of initial observations for training
training_size = 100
# Define the size of the rolling testing window
testing_size = 6

## potential Z's daily_volumes['Hourly Volume'],
Z = bond_prices.T

# Initialize the ARIMA-GARCH model, the Z is the name of the list that contains values to train the model
arima_model = pm.auto_arima(Z[:training_size], suppress_warnings=True)
garch_model = arch.arch_model(Z[:training_size], vol='Garch', p=1, q=1)

# Lists to store performance metrics
mae_list = []
mse_list = []
forecasts = []
log_returns = []


for i in range(2, len(Z)):
    previous_price = Z[i - 1]
    current_price = Z[i]
    log_return = math.log(current_price / previous_price)
    log_returns.append(log_return)

print(log_returns)
# Iterate through the data using ROCV
for i in range(training_size, len(Z) - testing_size + 1):

    training_data = log_returns[i - training_size:i]
    testing_data = log_returns[i:i + testing_size]

    # Re-estimate the ARIMA model with new training data
    arima_model.fit(training_data)
    best_arima_order = arima_model.order


    # Re-estimate the GARCH model with new training data

    garch_model = arch.arch_model(training_data, vol='Garch', p=best_arima_order[0], q=best_arima_order[1])
    res = garch_model.fit(disp='off')


    # Forecast the next value using the updated model
    forecast = arima_model.predict(n_periods=testing_size)
    volatility = res.conditional_volatility[-1]
    forecast = forecast * np.sqrt(volatility)
    forecast

    # Calculate evaluation metrics
    mae = mean_absolute_error(testing_data, forecast)
    mse = mean_squared_error(testing_data, forecast)

    mae_list.append(mae)
    mse_list.append(mse)

# Calculate the average or other statistics for the performance metrics
average_mae = np.mean(mae_list)
average_mse = np.mean(mse_list)


#Start at observation 100
#Iterate through entire list after item 100, at each step


[0.040188231919119397, 0.0001255121553865313, 0.00013067565724690148, -3.710774663254726e-05, 0.00011078658423706986, -1.4084515329523548e-06, -0.0001033296826553702, -0.00015735734222418967, 2.042302709334316e-06, 2.5969338049853957e-05, 6.189944908060991e-05, -2.0786483854000198e-05, -3.298252923728419e-05, 0.00021367060964371304, -0.00028340646318026665, 0.00016297908483004484, -5.80190624992488e-06, 6.300523701271697e-05, -2.4081654874220595e-05, 4.528332612918239e-05, -3.8486674095966564e-05, 0.0002049877171887462, -0.00021622997986133773, 3.362306360984353e-05, 3.652410475761684e-05, -3.5725713472385794e-05, -0.00014896064329439013, 8.010239991847564e-05, -6.473093399248336e-05, -6.172515365712154e-05, 0.00020478340020590191, -0.0001704327950717179, 0.00013184901448911644, -4.291489381096385e-06, -5.944329876105781e-05, 7.101704674169438e-05, 0.00012696454133610154, -0.00013327953812131588, 9.251585412613735e-05, -0.0001701752005211814, 0.0001204798583604303, -5.977593621396767e-

ValueError: ignored

In [146]:
#Asks

#Fix PL code based on new situation - Pranav

#Bring in time series code from DFAMM repo, train and generate 6 step ahead forecasts (at every hourly time period t) 63 * 6, 63 time periods we're using and at each time period
#we forecast 6 steps ahead for spread, price, and volume

#Calculate expected predicted loss at each time period based on the 6 steps ahead forecast data

#At each point Sum forecasted volume for the next 6 periods together, call this forecasted_total_volume, save dynamic_fee_percent = aggregate_forecasted_PL /  forecasted_total_volume

#Realized_fee_profit = prev period "actual" volume multiplied by prev period fee_percent, IFF the current price is within the range of LP position in the last period

#Chart fee_percent and realized_fee_profit

